# Sandbox

In [40]:
import requests
import pandas as pd
from datetime import datetime
import os

In [41]:
import math

In [42]:
from google.transit import gtfs_realtime_pb2

In [43]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two GPS coordinates using the Haversine formula.
    Args:
        lat1 (float): Latitude of the first coordinate.
        lon1 (float): Longitude of the first coordinate.
        lat2 (float): Latitude of the second coordinate.
        lon2 (float): Longitude of the second coordinate.
    Returns:
        float: Distance between the two coordinates in meters.
    """
    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Radius of the Earth in meters
    earth_radius = 6371000

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c

    return distance

# EXO

In [301]:
exo_file = 'exo_credentials.txt'

In [303]:
with open(exo_file, 'r') as f:
    line = f.read()

In [307]:
str(line.strip()).strip()

'9VZEYEXS3C'

In [83]:
exo_trips_url = 'https://opendata.exo.quebec/ServiceGTFSR/TripUpdate.pb?token=9VZEYEXS3C&agency=TRAINS'
exo_vehicle_url = 'https://opendata.exo.quebec/ServiceGTFSR/VehiclePosition.pb?token=9VZEYEXS3C&agency=TRAINS'

response = requests.get(exo_trips_url)
if response.status_code == 200:
    exo_trips = gtfs_realtime_pb2.FeedMessage()
    exo_trips.ParseFromString(response.content)
else:
    print(response.status_code)
    
response = requests.get(exo_vehicle_url)
if response.status_code == 200:
    exo_vehicles = gtfs_realtime_pb2.FeedMessage()
    exo_vehicles.ParseFromString(response.content)
else:
    print(response.status_code)

In [287]:
exo_trips = pd.read_csv(os.path.join('gtfs', 'exo', 'trips.txt'))
exo_routes = pd.read_csv(os.path.join('gtfs', 'exo', 'routes.txt'))

In [290]:
exo_trips.loc[exo_trips['trip_id'] == '4084038-TRAIN-H23-Blocks-Semaine-09']

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,trip_short_name,wheelchair_accessible,bikes_allowed
169,5,TRAIN-H23-Blocks-Semaine-09,4084038-TRAIN-H23-Blocks-Semaine-09,Lucien-L'Allier,0,50077,84,0,1


In [300]:
for exo_train in exo_vehicles.entity:
    #print(exo_train)
    
    if exo_train.HasField('vehicle'):
        vehicle = exo_train.vehicle
        print('Route: ' + vehicle.trip.route_id)
        print('Vehicle ID: ' + vehicle.vehicle.id)
        print('Trip ID: ' + vehicle.trip.trip_id)
        
        # resolve train no. from trips table
        try:
            condition = (exo_trips['trip_id'] == vehicle.trip.trip_id)
            print('Train No: ' + str(exo_trips.loc[condition, 'trip_short_name'].values[0]))
            print('Destination: ' + exo_trips.loc[condition, 'trip_headsign'].values[0])
        except:
            pass
        
        # resolve route name from routes table
        try:
            # because trip_id is numeric, pandas stores it as integer
            condition = (exo_routes['route_id'] == int(vehicle.trip.route_id))
            print('Route Name: ' + exo_routes.loc[condition, 'route_long_name'].values[0])
        except:
            pass
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        print('Distance from Gracey Side Rd: ' + str(round(calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)/10)/100) + ' km')
        print()
        

Route: 1
Vehicle ID: 3006
Trip ID: 4083546-TRAIN-H23-Blocks-Semaine-09
Train No: 13
Destination: Vaudreuil
Route Name: Vaudreuil–Hudson
Longitude: 45.401641845703125
Latitude: -73.958740234375
Distance from Gracey Side Rd: 766.27 km

Route: 5
Vehicle ID: 3011
Trip ID: 4084038-TRAIN-H23-Blocks-Semaine-09
Train No: 84
Destination: Lucien-L'Allier
Route Name: Candiac
Longitude: 45.42442321777344
Latitude: -73.6572494506836
Distance from Gracey Side Rd: 788.88 km



In [269]:
exo_train_list = []
for exo_trip in exo_trips.entity:
    #print(exo_trip)
    
    if exo_trip.HasField('trip_update'):
        trip_id = exo_trip.trip_update.trip.trip_id
        train_id = exo_trip.id
        exo_train_list.append({'train_id': train_id, 'trip_id': trip_id})
        
        print('Train ' + train_id + ' has trip id ' + trip_id)

print(exo_train_list)
print()

for exo_train in exo_vehicles.entity:
    #print(exo_train)
    
    if exo_train.HasField('vehicle'):
        vehicle = exo_train.vehicle
        print('Route: ' + vehicle.trip.route_id)
        print('Vehicle ID: ' + vehicle.vehicle.id)
        print('Trip ID: ' + vehicle.trip.trip_id)
        for exo_train in exo_train_list:
            if exo_train['trip_id'] == vehicle.trip.trip_id:
                print('Train No: ' + exo_train['train_id'])
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        print('Distance from Gracey Side Rd: ' + str(round(calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)/10)/100) + ' km')
        print()
        

Train 84 has trip id 4084038-TRAIN-H23-Blocks-Semaine-09
Train 184 has trip id 4084012-TRAIN-H23-Blocks-Semaine-09
Train 13 has trip id 4083546-TRAIN-H23-Blocks-Semaine-09
Train 22 has trip id 4083565-TRAIN-H23-Blocks-Semaine-09
[{'train_id': '84', 'trip_id': '4084038-TRAIN-H23-Blocks-Semaine-09'}, {'train_id': '184', 'trip_id': '4084012-TRAIN-H23-Blocks-Semaine-09'}, {'train_id': '13', 'trip_id': '4083546-TRAIN-H23-Blocks-Semaine-09'}, {'train_id': '22', 'trip_id': '4083565-TRAIN-H23-Blocks-Semaine-09'}]

Route: 1
Vehicle ID: 3006
Trip ID: 4083546-TRAIN-H23-Blocks-Semaine-09
Train No: 13
Longitude: 45.401641845703125
Latitude: -73.958740234375
Distance from Gracey Side Rd: 766.27 km

Route: 5
Vehicle ID: 3011
Trip ID: 4084038-TRAIN-H23-Blocks-Semaine-09
Train No: 84
Longitude: 45.42442321777344
Latitude: -73.6572494506836
Distance from Gracey Side Rd: 788.88 km



# VIA

In [25]:
via_feed_url = 'https://asm-backend.transitdocs.com/gtfs/via'
amtrak_feed_url = 'https://asm-backend.transitdocs.com/gtfs/amtrak'

response = requests.get(via_feed_url)
if response.status_code == 200:
    via_feed = gtfs_realtime_pb2.FeedMessage()
    via_feed.ParseFromString(response.content)

    # Process the GTFS Realtime data
    # You can access various entities in the feed, such as feed.entity, feed.header, etc.
else:
    print('Error retrieving GTFS Realtime data:', response.status_code)
    
response = requests.get(amtrak_feed_url)
if response.status_code == 200:
    amtrak_feed = gtfs_realtime_pb2.FeedMessage()
    amtrak_feed.ParseFromString(response.content)

    # Process the GTFS Realtime data
    # You can access various entities in the feed, such as feed.entity, feed.header, etc.
else:
    print('Error retrieving GTFS Realtime data:', response.status_code)

In [44]:
via_trips = pd.read_csv(os.path.join('gtfs', 'viarail', 'trips.txt'))
via_routes = pd.read_csv(os.path.join('gtfs', 'viarail', 'routes.txt'))

In [4]:
via_routes

,route_id,route_short_name,route_long_name,route_type,route_color,route_text_color,agency_id
0,119-93,VIA Rail,Toronto - London,2,FFCB06,0,1
1,119-58,VIA Rail,Toronto - Kingston,2,FFCB06,0,1
2,226-460,VIA Rail,Montréal - Senneterre,2,FFCB06,0,1
3,226-119,VIA Rail,Montréal - Toronto,2,FFCB06,0,1
4,226-444,VIA Rail,Montréal - Jonquière,2,FFCB06,0,1
5,8-119,VIA Rail,Vancouver - Toronto,2,FFCB06,0,1
6,119-341,VIA Rail,Toronto - Sarnia,2,FFCB06,0,1
7,621-116,VIA Rail,Sudbury - White River,2,FFCB06,0,1
8,388-435,VIA Rail,Winnipeg - Churchill,2,FFCB06,0,1
9,617-628,VIA Rail,Ottawa - Québec,2,FFCB06,0,1


In [75]:
via_feed_url = 'https://asm-backend.transitdocs.com/gtfs/via'
response = requests.get(via_feed_url)
if response.status_code == 200:
    via_feed = gtfs_realtime_pb2.FeedMessage()
    via_feed.ParseFromString(response.content)

In [49]:
train_list = []
for via_train in via_feed.entity:
    #print(via_train)
    
    if via_train.HasField('vehicle'):
        vehicle = via_train.vehicle
        print(vehicle)
        
        print('Trip Start Date: ' + vehicle.trip.start_date)
        timedelta = datetime.now() - datetime.fromtimestamp(vehicle.timestamp)
        
        if timedelta.seconds > 90:
            #print(f"data is {timedelta.seconds} too old, skipping")
            print(timedelta, timedelta.seconds)
            continue
        else:
            print(timedelta, timedelta.seconds)
        
        
        print('Updated: ' + datetime.fromtimestamp(vehicle.timestamp).strftime('%Y-%m-%d %H:%M:%S') + ', ' + str(timedelta) + ' ago.')
        
        # resolve route
        try:
            condition = via_routes['route_id'] == vehicle.trip.route_id
            print('Route: ' + str(via_routes.loc[condition, 'route_long_name'].values[0]))
        except Exception as ex:
            print(ex)
        
        # resolve trip
        try:
            # because trip_id is numeric, pandas stores it as integer
            condition = (via_trips['trip_id'] == int(vehicle.trip.trip_id)) & (via_trips['route_id'] == vehicle.trip.route_id)
            print('Train Number (Trip Short Name): ' + via_trips.loc[condition, 'trip_short_name'].values[0])
            print('Destination: ' + via_trips.loc[condition, 'trip_headsign'].values[0])
        except:
            pass
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        dst = calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)
        print('Distance from Gracey Side Rd: ' + str(round(dst/10)/100) + ' km')
        
        # https://asm.transitdocs.com/train/2023/5/29/V/63
        print(f'https://asm.transitdocs.com/train/{vehicle.trip.start_date[0:4]}/{vehicle.trip.start_date[4:6]}/{vehicle.trip.start_date[6:8]}/V/{vehicle.vehicle.id[15:]}')
        
        print()

trip {
  trip_id: "525"
  route_id: "388-435"
  direction_id: 0
  start_date: "20230604"
  schedule_relationship: SCHEDULED
}
vehicle {
  id: "2023-06-04_VIA_693"
}
position {
  latitude: 49.983
  longitude: -98.3545
  speed: 20.2777348
}
current_stop_sequence: 3
stop_id: "104"
current_status: IN_TRANSIT_TO
timestamp: 1685902937

Trip Start Date: 20230604
0:03:56.538466 236
trip {
  trip_id: "270"
  route_id: "621-116"
  direction_id: 1
  start_date: "20230604"
  schedule_relationship: SCHEDULED
}
vehicle {
  id: "2023-06-04_VIA_186"
}
position {
  latitude: 47.5133
  longitude: -82.6352
  bearing: 116
  speed: 7.22416639
}
current_stop_sequence: 21
stop_id: "265"
current_status: IN_TRANSIT_TO
timestamp: 1685901900

Trip Start Date: 20230604
0:21:13.538466 1273
trip {
  trip_id: "539"
  route_id: "628-576"
  direction_id: 1
  start_date: "20230604"
  schedule_relationship: SCHEDULED
}
vehicle {
  id: "2023-06-04_VIA_24"
}
position {
  latitude: 46.1289
  longitude: -72.3367
  speed: 35

# Amtrak

In [163]:
amtrak_feed_url = 'https://asm-backend.transitdocs.com/gtfs/amtrak'
response = requests.get(amtrak_feed_url)
if response.status_code == 200:
    amtrak_feed = gtfs_realtime_pb2.FeedMessage()
    amtrak_feed.ParseFromString(response.content)

In [193]:
for amtk_train in amtrak_feed.entity:
    #print(entity)
    
    if amtk_train.HasField('vehicle'):
        vehicle = amtk_train.vehicle
        #print(vehicle)
        
        print('Trip Start Date: ' + vehicle.trip.start_date)
        print('Updated: ' + datetime.fromtimestamp(vehicle.timestamp).strftime('%Y-%m-%d %H:%M:%S'))
        
        #condition = (amtrak_trips['trip_id'] == vehicle.trip.trip_id) & (amtrak_trips['route_id'] == vehicle.trip.route_id)
        #print('Train ID: ' + amtrak_trips.loc[condition, 'trip_short_name'].values[0])
        #print('Destination: ' + amtrak_trips.loc[condition, 'trip_headsign'].values[0])
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        print('Distance from Gracey Side Rd: ' + str(round(calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)/1000)) + ' km')
        print()

Trip Start Date: 20230529
Updated: 2023-05-29 23:57:08
Longitude: 43.19765853881836
Latitude: -70.87969970703125
Distance from Gracey Side Rd: 954 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:57:06
Longitude: 37.22612762451172
Latitude: -120.2425537109375
Distance from Gracey Side Rd: 3247 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:34
Longitude: 37.293643951416016
Latitude: -120.32691192626953
Distance from Gracey Side Rd: 3251 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:07
Longitude: 37.93414306640625
Latitude: -121.3546142578125
Distance from Gracey Side Rd: 3310 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:02
Longitude: 41.34831619262695
Latitude: -71.87882232666016
Distance from Gracey Side Rd: 886 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:57
Longitude: 40.326637268066406
Latitude: -74.61068725585938
Distance from Gracey Side Rd: 695 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:43:00
Longitude: 40.752464294433594
Lat

# Generic function    

In [93]:
import os
def get_trains(agency_feed = False, agency_name = ''):

    # initialize empty train list
    nearby_trains = []
    rejected_trains = []

    # Check for valif agency
    agencies = ['VIA', 'exo']

    if not agency_feed or not (agency_name in agencies):
        log.error(f'get_trains: no agency feed or unknown agency name: {agency_name}')
        return nearby_trains

    #2 get camera coordinates
    lon = -73.54208
    lat = 45.48704

    if (lon == 0) or (lat == 0):
        print(f'find_closest_train: skipping, location of camera 000 is not set.')
        return nearby_trains
    else:
        print(f'find_closest_train: comparing train positions with camera location {lat}, {lon}.')

    # Read trips and routes from static GTFS
    try:
        if agency_name == 'VIA':
            trips = pd.read_csv(os.path.join('gtfs', 'viarail', 'trips.txt'))
            routes = pd.read_csv(os.path.join('gtfs', 'viarail', 'routes.txt'))
        elif agency_name == 'exo':
            trips = pd.read_csv(os.path.join('gtfs', 'exo', 'trips.txt'))
            routes = pd.read_csv(os.path.join('gtfs', 'exo', 'routes.txt'))

    except Exception as ex:
        trips = False
        routes = False
        print('find_closest_train0: ' + str(ex))

    # loop through trains:
    for train in agency_feed.entity:

        if train.HasField('vehicle'):
            vehicle = train.vehicle

            # age of record
            timedelta = datetime.now() - datetime.fromtimestamp(vehicle.timestamp)  # fix1

            # distance:
            dst = calculate_distance(lat, lon, vehicle.position.latitude, vehicle.position.longitude)
            dst_str = str(round(dst)) # distance in m

            # initialize result string
            train_str = ''
            train_no = '?'
            train_destination = ''
            route_name = ''

            # resolve train no. from trips table
            try:
                # because route_id is numeric, pandas stores it as integer
                try:
                    trip_id = int(vehicle.trip.trip_id)
                except:
                    trip_id = vehicle.trip.trip_id

                print(f'Trying to resolve {trip_id}')
                condition = (trips['trip_id'] == trip_id)
                train_no = str(trips.loc[condition, 'trip_short_name'].values[0])
                train_destination = trips.loc[condition, 'trip_headsign'].values[0]
            except Exception as ex:
                print(f'find_closest_train1: looking for trip_id {trip_id} in {trips}' + str(ex))

            # resolve route name from routes table
            try:
                # because route_id is numeric, pandas stores it as integer
                try:
                    route_id = int(vehicle.trip.route_id)
                except:
                    route_id = vehicle.trip.route_id

                condition = (routes['route_id'] == route_id)
                route_name = routes.loc[condition, 'route_long_name'].values[0]
            except Exception as ex:
                print('find_closest_train2: ' + str(ex))


            # assign vehicle id and route number
            try:
                if agency_name == 'VIA':
                    try:
                        asm_url = f'https://asm.transitdocs.com/train/{vehicle.trip.start_date[0:4]}/{vehicle.trip.start_date[4:6]}/{vehicle.trip.start_date[6:8]}/V/{vehicle.vehicle.id[15:]}'
                    except:
                        asm_url = ''
                else:
                    asm_url = ''

                if agency_name == 'exo':
                    vehicle_id = vehicle.vehicle.id
                    train_str = f'EXO {vehicle_id} on '

                # build train string
                train_str += f'{agency_name} {train_no}, route: {route_name}, destination: {train_destination}'
                if (len(asm_url) > 0) and agency_name == 'VIA':
                    train_str += f', details: {asm_url}'

                # log
                print(f'find_closest_train: found train {train_str}, at distance: {dst_str} m, age: {timedelta.seconds} s')
            except Exception as ex:
                print(f'find_closest_train3: ' + str(ex))
                train_str = vehicle.vehicle.id

            # check distance and time limit
            if (dst <= 1000) and (timedelta.seconds <= 120):
                nearby_trains.append(train_str)
            else:
                rejected_trains.append(train_str)

    return nearby_trains

In [94]:
get_trains(agency_name='VIA', agency_feed=via_feed)

find_closest_train: comparing train positions with camera location 45.48704, -73.54208.
Trying to resolve 452
find_closest_train: found train VIA 668, route: Montréal - Toronto, destination: Montréal, details: https://asm.transitdocs.com/train/2023/06/04/V/668, at distance: 427003 m, age: 445 s
Trying to resolve 456
find_closest_train: found train VIA 669, route: Montréal - Toronto, destination: Toronto, details: https://asm.transitdocs.com/train/2023/06/04/V/669, at distance: 39614 m, age: 444 s
Trying to resolve 525
find_closest_train: found train VIA 693, route: Winnipeg - Churchill, destination: Churchill, details: https://asm.transitdocs.com/train/2023/06/04/V/693, at distance: 2091788 m, age: 516 s
Trying to resolve 2023-06-04_VIA_26
find_closest_train1: looking for trip_id 2023-06-04_VIA_26 in    route_id  service_id  trip_id  shape_id trip_short_name trip_headsign  \
0   119-341         498      498       498              85        London   
1   119-341         499      499    

[]

In [84]:
get_trains(agency_name = 'exo', agency_feed = exo_vehicles)

find_closest_train: comparing train positions with camera location 45.48704, -73.54208.
find_closest_train: found train EXO 2003 on train exo 209, route: Saint-Jérôme, destination: Saint-Jérôme, at distance: 44876 m, age: 18 s
find_closest_train: found train EXO 3012 on train exo 64, route: Vaudreuil–Hudson, destination: Lucien-L'Allier, at distance: 25730 m, age: 22 s


[]